In [1]:

# Imports
import os
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
from scipy import signal
from scipy.io import wavfile
from io import BytesIO
from IPython.display import Audio, display
%matplotlib inline

def load_signal(upload_widget, file_path, fs_manual):
    """Charge un signal WAV ou Excel, utilise fs_manual pour Excel."""
    # Essai via chemin
    path = file_path.strip() if isinstance(file_path, str) else ''
    if path and os.path.isfile(path):
        ext = os.path.splitext(path)[1].lower()
        if ext == '.wav':
            fs, x = wavfile.read(path)
            x = x[:,0] if x.ndim==2 else x
            return x.astype(float), fs
        if ext in ('.xlsx','xls'):
            df = pd.read_excel(path, header=None)
            return df.iloc[:,0].values, fs_manual
    # Fallback upload widget
    if upload_widget.value:
        for meta, content in upload_widget.value.items():
            name = meta['name']
            bio = BytesIO(content)
            ext = os.path.splitext(name)[1].lower()
            if ext == '.wav':
                fs, x = wavfile.read(bio)
                x = x[:,0] if x.ndim==2 else x
                return x.astype(float), fs
            if ext in ('.xlsx','xls'):
                df = pd.read_excel(bio, header=None)
                return df.iloc[:,0].values, fs_manual
    raise ValueError("Aucun fichier valide disponible.")


In [2]:

def process_signal(x, fs, module,
                   facteur_ech, bits,
                   type_fenetre, type_filtre,
                   f_basse, f_haute, ordre, zp_factor):
    """Applique le module choisi au signal x."""
    sig = x.copy()
    fs2 = fs
    if module=='Échantillonnage':
        Nnew = max(1, int(len(sig)*facteur_ech))
        sig = signal.resample(sig, Nnew)
        fs2 = int(fs*facteur_ech)
        if bits>0:
            levels=2**bits
            mn,mx=sig.min(),sig.max()
            norm=(sig-mn)/(mx-mn)
            sig=(np.round(norm*(levels-1))/(levels-1))*(mx-mn)+mn
    elif module=='Fenêtrage':
        win = signal.get_window(type_fenetre, len(sig))
        sig *= win
    elif module=='Filtrage':
        nyq=fs2/2.0
        ntaps = ordre + 1
        if type_filtre in ('Passe-bande', 'Coupe-bande') and (ntaps % 2 == 0):
            ntaps += 1
        lb=max(f_basse,1e-6); hb=min(f_haute,nyq-1e-6)
        if hb<=lb: hb=lb+1e-6
        if type_filtre=='Passe-bas':
            taps=signal.firwin(ntaps, hb/nyq, pass_zero=True)
        elif type_filtre=='Passe-haut':
            taps=signal.firwin(ntaps, lb/nyq, pass_zero=False)
        elif type_filtre=='Passe-bande':
            taps=signal.firwin(ntaps, [lb/nyq,hb/nyq], pass_zero=False)
        elif type_filtre=='Coupe-bande':
            taps=signal.firwin(ntaps, [lb/nyq,hb/nyq], pass_zero=True)
        sig = signal.filtfilt(taps,1.0,sig)
    elif module=='Zero padding':
        pad_len=int((zp_factor-1)*len(sig))
        if pad_len>0:
            sig=np.pad(sig,(0,pad_len),mode='constant')
    return sig, fs2


In [3]:

def plot_comparison(upload_w, file_path, fs_manual,
                    module, facteur_ech, bits,
                    type_fenetre, type_filtre,
                    f_basse, f_haute, ordre, zp_factor,
                    t_debut, t_fin, f_debut, f_fin):
    import numpy as np
    import matplotlib.pyplot as plt
    from scipy import signal
    from IPython.display import Audio, display

    # Charger et traiter
    x, fs = load_signal(upload_w, file_path, fs_manual)
    y, fs2 = process_signal(x, fs, module,
                             facteur_ech, bits,
                             type_fenetre, type_filtre,
                             f_basse, f_haute, ordre, zp_factor)

    # FFT domaines
    N1, N2 = len(x), len(y)
    t1 = np.arange(N1) / fs
    t2 = np.arange(N2) / fs2
    f1 = np.fft.rfftfreq(N1, 1/fs)
    A1 = (2.0/N1) * np.abs(np.fft.rfft(x))
    f2 = np.fft.rfftfreq(N2, 1/fs2)
    A2 = (2.0/N1) * np.abs(np.fft.rfft(y))

    # Masques
    m_t1 = (t1 >= t_debut) & (t1 <= t_fin)
    m_t2 = (t2 >= t_debut) & (t2 <= t_fin)
    m_f1 = (f1 >= f_debut) & (f1 <= f_fin)
    m_f2 = (f2 >= f_debut) & (f2 <= f_fin)

    # Domaine temps & FFT
    fig1, axs1 = plt.subplots(1,2, figsize=(12,4))
    # Temps
    axs1[0].plot(t1[m_t1], x[m_t1], label='Original', color='C0', linestyle='-')
    if module == 'Échantillonnage':
        axs1[0].scatter(t2[m_t2], y[m_t2], label='Échantillonné', color='C1', s=10)
    else:
        axs1[0].plot(t2[m_t2], y[m_t2], label='Modifié', color='C1', linestyle='--')
    axs1[0].set_title('Domaine temps')
    axs1[0].set_xlabel('Temps (s)'); axs1[0].set_ylabel('Amplitude')
    axs1[0].legend(); axs1[0].grid(True)
    # FFT
    axs1[1].plot(f1[m_f1], A1[m_f1], label='Original', color='C0', linestyle='-')
    axs1[1].plot(f2[m_f2], A2[m_f2], label='Modifié', color='C1', linestyle='--')
    axs1[1].set_title('Domaine fréquence')
    axs1[1].set_xlabel('Fréquence (Hz)'); axs1[1].set_ylabel('Amplitude spectrale')
    axs1[1].legend(); axs1[1].grid(True)
    fig1.tight_layout()
    plt.show()

    # Spectrogrammes côte-à-côte
    Nperseg = min(256, N1, N2)
    noverlap = Nperseg // 2
    fsp1, tsp1, S1 = signal.spectrogram(x, fs, window='hann', nperseg=Nperseg, noverlap=noverlap)
    fsp2, tsp2, S2 = signal.spectrogram(y, fs2, window='hann', nperseg=Nperseg, noverlap=noverlap)
    fig2, axs2 = plt.subplots(1,2, figsize=(12,4))
    pcm1 = axs2[0].pcolormesh(tsp1, fsp1, 10*np.log10(S1+1e-12), shading='gouraud')
    axs2[0].set_title('Spectrogramme Original')
    axs2[0].set_xlabel('Temps (s)'); axs2[0].set_ylabel('Fréquence (Hz)')
    if len(tsp1) > 1: axs2[0].set_xlim(t_debut, t_fin)
    if len(fsp1) > 1: axs2[0].set_ylim(f_debut, f_fin)
    pcm2 = axs2[1].pcolormesh(tsp2, fsp2, 10*np.log10(S2+1e-12), shading='gouraud')
    axs2[1].set_title('Spectrogramme Modifié')
    axs2[1].set_xlabel('Temps (s)'); axs2[1].set_ylabel('Fréquence (Hz)')
    if len(tsp2) > 1: axs2[1].set_xlim(t_debut, t_fin)
    if len(fsp2) > 1: axs2[1].set_ylim(f_debut, f_fin)
    fig2.tight_layout()
    fig2.colorbar(pcm1, ax=axs2, label='Puissance (dB)')
    plt.show()

    # Lecture audio
    ext = os.path.splitext(file_path)[1].lower()
    if ext == '.wav':
        display(Audio(data=y, rate=fs2))


In [4]:

import ipywidgets as widgets
from ipywidgets import interactive_output

# Exemples
files=[f for f in os.listdir('data') if os.path.isfile(os.path.join('data',f))]
ex_w=widgets.Dropdown(options=files,description='Exemple:')
file_w=widgets.Text(description='Chemin:',value=f"data/{files[0]}" if files else '')
upload_w=widgets.FileUpload(description='Charger')
fs_w=widgets.FloatText(description='Fs (Hz)',value=1e6)

# Modules
mod_w=widgets.Dropdown(options=['Échantillonnage','Fenêtrage','Filtrage','Zero padding'],description='Module:')
# Params
fact_w=widgets.FloatSlider(description='Taux x',min=0.01,max=2.0,step=0.01,value=1.0)
bits_w=widgets.IntSlider(description='Bits',min=0,max=24,value=16)
win_w=widgets.Dropdown(options=['hann','hamming','boxcar','blackman'],description='Fenêtre:')
filt_w=widgets.Dropdown(options=['Passe-bas','Passe-haut','Passe-bande','Coupe-bande'],description='Filtre:')
fb_w=widgets.FloatSlider(description='F basse',min=0,max=200000,step=100,value=1000)
fh_w=widgets.FloatSlider(description='F haute',min=0,max=200000,step=100,value=5000)
ord_w=widgets.IntSlider(description='Ordre',min=1,max=128,value=8)
zp_w=widgets.FloatSlider(description='ZP fact',min=1,max=10,step=0.5,value=1)

t0_w=widgets.FloatSlider(description='t0',min=0,max=1,step=0.01,value=0)
t1_w=widgets.FloatSlider(description='t1',min=0,max=1,step=0.01,value=1)
f0_w=widgets.IntSlider(description='f0',min=0,max=200000,step=10,value=0)
f1_w=widgets.IntSlider(description='f1',min=0,max=200000,step=10,value=200000)

# Module toggle
params=[fact_w,bits_w,win_w,filt_w,fb_w,fh_w,ord_w,zp_w]
for p in params: p.layout.display='none'
def on_mod(c):
    for p in params: p.layout.display='none'
    if c['new']=='Échantillonnage': fact_w.layout.display=bits_w.layout.display=''
    elif c['new']=='Fenêtrage': win_w.layout.display=''
    elif c['new']=='Filtrage':
        filt_w.layout.display=fb_w.layout.display=fh_w.layout.display=ord_w.layout.display=''
    elif c['new']=='Zero padding': zp_w.layout.display=''
mod_w.observe(on_mod,names='value'); on_mod({'new':mod_w.value})

# Exemple callback
def on_ex(c): file_w.value=f"data/{c['new']}"
ex_w.observe(on_ex,names='value')

# Dynamic sliders
def up_sl(*a):
    try:
        x,fs=load_signal(upload_w,file_w.value,fs_w.value)
    except: return
    duration=len(x)/fs; step_t=duration/1000.0; 
    if step_t > 0:
        decimals_t = max(0, -int(math.floor(math.log10(step_t))))
    else:
        decimals_t = 1
    fmt_time = f".{decimals_t}f"

    for w in (t0_w, t1_w):
        w.max = duration
        w.step = step_t
        if w.value > duration:
            w.value = duration if w is t1_w else 0
        w.readout_format = fmt_time
    fmax=fs/2.0
    for w in (f0_w,f1_w):
        w.max = fmax
        w.step = 10
        if w.value>fmax: w.value=fmax if w is f1_w else 0
        w.readout_format = 'd'
        
for w in (upload_w,file_w,fs_w,ex_w): w.observe(up_sl,names='value')
up_sl()

out=interactive_output(plot_comparison,{
    'upload_w':upload_w,'file_path':file_w,'fs_manual':fs_w,
    'module':mod_w,'facteur_ech':fact_w,'bits':bits_w,'type_fenetre':win_w,
    'type_filtre':filt_w,'f_basse':fb_w,'f_haute':fh_w,'ordre':ord_w,'zp_factor':zp_w,
    't_debut':t0_w,'t_fin':t1_w,'f_debut':f0_w,'f_fin':f1_w
})

display(widgets.VBox([
    widgets.HBox([ex_w,file_w,upload_w,fs_w]),
    mod_w,
    widgets.HBox([fact_w,bits_w]),
    win_w,
    widgets.HBox([filt_w,fb_w,fh_w,ord_w]),
    zp_w,
    widgets.HBox([t0_w,t1_w,f0_w,f1_w]),
    out
]))
